In [30]:
import numpy as np
from pathlib import Path
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as f
import torch.nn as nn
from PIL import Image
from torchvision.transforms import ToTensor
from models.vae_dataset import VAENpDataset
from models.vae import ResVAE
from torch import optim
from models.actor import ActorCritic, BackboneNetwork
from tqdm import tqdm

In [31]:
dataset = VAENpDataset(path_to_np_files="data", img_size=96)

100%|██████████| 14/14 [00:00<00:00, 14.04it/s]


In [32]:
def load_checkpoint(model, optimizer, path, device):
    checkpoint = torch.load(path, map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    loss = checkpoint['loss']
    print(f'Checkpoint loaded: {path}, resuming from epoch {start_epoch}')
    return start_epoch, loss

In [33]:
resvae = ResVAE(3, hidden_dim=4096, z_dim=32)
optimizer = optim.Adam(resvae.parameters(), lr=0.00001)

In [34]:
_, _ = load_checkpoint(resvae, optimizer, './checkpoints/checkpoint_epoch_20.pt', "cpu")

Checkpoint loaded: ./checkpoints/checkpoint_epoch_20.pt, resuming from epoch 20


In [35]:
x = dataset[0]

In [40]:
x.shape

torch.Size([3, 96, 96])

In [38]:
z, _, _ = resvae.encode(x.unsqueeze(0))

In [39]:
z.shape

torch.Size([1, 32])

In [11]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from PIL import Image

In [12]:
def create_vae_animation(npz_path, vae_model, save_path="vae_reconstruction.gif", num_frames=400):
    """
    npz_path: путь к файлу с данными (предполагаем ключ 'obs')
    vae_model: ваша загруженная модель (torch.nn.Module)
    save_path: куда сохранить гифку
    num_frames: сколько кадров анимировать
    """

    # 1. Загрузка данных
    print(f"Загрузка данных из {npz_path}...")
    data = np.load(npz_path)

    # Предполагаем, что картинки лежат по ключу 'obs' или 'observations'
    # Если у вас другой ключ, поменяйте его здесь
    key = 'obs' if 'obs' in data else list(data.keys())[0]
    observations = data[key] # Ожидаем форму (N, 96, 96, 3) или (N, 64, 64, 3)

    # 2. Подготовка модели
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    vae_model.to(device)
    vae_model.eval() # Переключаем в режим оценки (выключает dropout и т.д.)

    # 3. Настройка графика
    fig, ax = plt.subplots()
    ax.axis('off') # Убираем оси координат

    # Инициализация пустой картинки
    # Берем первый кадр просто чтобы узнать размер
    img_plot = ax.imshow(observations[0])

    print("Генерация анимации...")

    def update(frame_idx):
        # --- ПРЕПРОЦЕССИНГ ---
        # Берем кадр из датасета. Он сейчас (H, W, C) и 0-255 uint8
        raw_frame = observations[frame_idx]

        # Конвертируем в формат PyTorch: (1, C, H, W) и нормализуем 0-1
        # permute(2, 0, 1) делает из HWC -> CHW
        input_tensor = torch.from_numpy(raw_frame).permute(2, 0, 1).float() / 255.0

        # Добавляем Batch dimension -> (1, C, H, W)
        x = input_tensor.unsqueeze(0).to(device)

        # --- ВАШ КОД + ИНФЕРЕНС ---
        with torch.no_grad(): # Отключаем градиенты для скорости
            # ВАШ СНИППЕТ ВСТАВЛЕН СЮДА
            # x подается в VAE, на выходе реконструированная картинка

            # Примечание: Ваш сниппет возвращает PIL Image.
            # Matplotlib хочет numpy array. Поэтому оборачиваем в np.array()
            reconstructed_pil = Image.fromarray(
                (resvae(x)[0].squeeze(0).cpu().detach().numpy().transpose(1,2,0).astype(np.float32)*255).astype(np.uint8)
            )

            reconstructed_img = np.array(reconstructed_pil)

        # Обновляем картинку на графике
        img_plot.set_data(reconstructed_img)
        ax.set_title(f"Frame: {frame_idx}")
        return [img_plot]

    # Создание анимации
    # frames=num_frames ограничивает длину, чтобы не рендерить весь датасет часами
    ani = FuncAnimation(fig, update, frames=range(num_frames), blit=True)

    # Сохранение
    print(f"Сохранение в {save_path}...")
    ani.save(save_path, fps=20, writer='pillow') # 'pillow' сохраняет gif
    print("Готово!")
    plt.close()

In [13]:
path_to_npz = "data/car_racing_data_ep_12.npz" # Или "data_1.npz"

# 2. Здесь должна быть инициализация вашей модели

# ПРОВЕРКА: Если модели нет под рукой, код упадет.
    # Раскомментируйте строку ниже для запуска функции, если vae уже определен в вашем коде
create_vae_animation(path_to_npz, resvae)

Загрузка данных из data/car_racing_data_ep_12.npz...
Генерация анимации...
Сохранение в vae_reconstruction.gif...
Готово!


In [19]:
def calculate_returns(rewards, discount_factor):
    returns = []
    cumulative_reward = 0
    for r in reversed(rewards):
        cumulative_reward = r + cumulative_reward * discount_factor
        returns.insert(0, cumulative_reward)
    returns = torch.tensor(returns)
    # normalize the return
    returns = (returns - returns.mean()) / returns.std()
    return returns

In [20]:
def calculate_advantages(returns, values):
    advantages = returns - values
    # Normalize the advantage
    advantages = (advantages - advantages.mean()) / advantages.std()
    return advantages

In [21]:
def calculate_surrogate_loss(
        actions_log_probability_old,
        actions_log_probability_new,
        epsilon,
        advantages):
    advantages = advantages.detach()
    policy_ratio = (
            actions_log_probability_new - actions_log_probability_old
            ).exp()
    surrogate_loss_1 = policy_ratio * advantages
    surrogate_loss_2 = torch.clamp(
            policy_ratio, min=1.0-epsilon, max=1.0+epsilon
            ) * advantages
    surrogate_loss = torch.min(surrogate_loss_1, surrogate_loss_2)
    return surrogate_loss

In [24]:
def calculate_losses(
        surrogate_loss, entropy, entropy_coefficient, returns, value_pred):
    entropy_bonus = entropy_coefficient * entropy
    policy_loss = -(surrogate_loss + entropy_bonus).sum()
    value_loss = f.smooth_l1_loss(returns, value_pred).sum()
    return policy_loss, value_loss

In [25]:
import gymnasium as gym
import pygame
import numpy as np

/Users/daniilogorodnikov/PycharmProjects/DeepWorld/.venv/lib/python3.13/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [26]:
env = gym.make("CarRacing-v3", render_mode="human")

In [27]:
def create_agent(hidden_dimensions, dropout):
    INPUT_FEATURES = env.observation_space.shape[0]
    HIDDEN_DIMENSIONS = hidden_dimensions
    ACTOR_OUTPUT_FEATURES = env.action_space.n
    CRITIC_OUTPUT_FEATURES = 1
    DROPOUT = dropout
    actor = BackboneNetwork(
            INPUT_FEATURES, HIDDEN_DIMENSIONS, ACTOR_OUTPUT_FEATURES, DROPOUT)
    critic = BackboneNetwork(
            INPUT_FEATURES, HIDDEN_DIMENSIONS, CRITIC_OUTPUT_FEATURES, DROPOUT)
    agent = ActorCritic(actor, critic)
    return agent

In [28]:
def init_training():
    states = []
    actions = []
    actions_log_probability = []
    values = []
    rewards = []
    done = False
    episode_reward = 0
    return states, actions, actions_log_probability, values, rewards, done, episode_reward